#  AAS JSON →  Knowledge GraphConversion Pipeline

This notebook demonstrates the transformation of:
- an **ISO 63278 Asset Administration Shell (AAS) JSON** 
- via a **SPARQL CONSTRUCT mapping**  
- into  **OWL-based Knowledge Graph (KG)**  

Pipeline Steps:

1. Load AAS instance
2. Validate JSON against AAS Meta-Model schema
3. Convert AAS JSON -> RDF (using `py-aas-rdf`)
4. Apply SPARQL mapping → AAS RDF
5. Save output RDF



## Imports & Environment Setup

In [5]:
import json
from rdflib import Graph, Namespace
from py_aas_rdf.models.submodel import Submodel
from pyoxigraph import RdfFormat, Store
import rdflib
from pathlib import Path
from dsms import DSMS, KItem
from pyshacl import validate


## Variable initialization and load configuration from .env to initialize DSMS instance

In [6]:
# -----------------------------
# Paths & constants
# -----------------------------
AAS_JSON_PATH = Path("./input/InspectionDocumentsOfSteelProducts.json")
INSPECTION_DOCUMENT_MAPPING_PATH = Path("./input/InspectionDocumentsOfSteelProducts_AAS2KG_mapping.sparql")
OUTPUT_DIR = Path("./output")

PLACEHOLDER = "PLACEHOLDER"
UNIQUE_ID = "IDOSP123"

AAS_NS = Namespace("https://admin-shell.io/aas/3/0/")
BASE_URI = "https://example.org/aas/"

# DSMS setup
dsms = DSMS(env=".env")

[2026-02-02 20:09:23,949 - dsms.core.configuration - INFO]: Authenticated with User name and Password at https://pmdx.materials-data.space/


## Apply AAS2KG pipeline

In [7]:

# Load AAS JSON
with open(AAS_JSON_PATH, "r", encoding="utf-8") as f:
    aas_json = json.load(f)
        
# Run for each submodel in inspection document
for sm_json in aas_json.get("submodels", []):
    rdf_graph = Graph()
    rdf_graph.bind("aas", "https://admin-shell.io/aas/3/0/")
    
    # Validate submodel using py-aas-rdf library
    submodel = Submodel.model_validate(sm_json)
    # Convert submodel to RDF
    submodel.to_rdf(rdf_graph, base_uri="https://example.org/aas/",  id_strategy="base64-url-encode")
    ttl_output = rdf_graph.serialize(format="turtle")
    
    # Apply SPARQL construct for specific submodels 
    if sm_json.get("idShort") == "InspectionDocumentsOfSteelProducts":
        with open(INSPECTION_DOCUMENT_MAPPING_PATH, "r", encoding="utf-8") as f:
            mapping_query = f.read().replace(PLACEHOLDER, UNIQUE_ID)

            store = Store()

            store.load(ttl_output, format=RdfFormat.TURTLE)
            mapping_result = store.query(mapping_query)

            # Serialize the result from Oxigraph (returns bytes)
            mapped_bytes = mapping_result.serialize(format=RdfFormat.TURTLE)

            # --- Parse into rdflib.Graph for prefix binding ---
            g = rdflib.Graph()
            g.parse(data=mapped_bytes, format="turtle")

            g.bind("aas", "https://admin-shell.io/aas/3/0/")
            g.bind("ex", "http://www.example.org/#")
            g.bind("pmd", "https://w3id.org/pmd/co")
            g.bind("obo", "http://purl.obolibrary.org/obo")

            # Serialize with prefixes applied
            final_ttl = g.serialize(format="turtle")
            
            print("===============InspectionDocumentsOfSteelProducts=======================")
            print("✓ Applied SPARQL mapping and namespace bindings.")
            print(final_ttl)
            # Write to file
            with open("./output/InspectionDocumentsOfSteelProducts_AAS.ttl", "w", encoding="utf-8") as f:
                f.write(final_ttl)

            
            
            print("✓ Applied SHACL shape for chemical composition")
            shapes = Graph().parse("./input/chemical_composition_shape.ttl")
            conforms, report_graph, report_text = validate(
                data_graph=final_ttl,
                shacl_graph=shapes,
            )
            print(report_text)

===============InspectionDocumentsOfSteelProducts=======================
✓ Applied SPARQL mapping and namespace bindings.
@prefix ex: <http://www.example.org/#> .
@prefix ns1: <https://w3id.org/pmd/co/> .
@prefix ns2: <http://purl.obolibrary.org/obo/> .

ex:IDOSP123_chem_comp a ns1:PMD_0000551 ;
    ns2:RO_0000080 ex:IDOSP123_material ;
    ns1:PMD_0000004 ex:IDOSP123_chem_comp_spec .

ex:IDOSP123_mass_proportion_carbon a ns1:PMD_0020102 ;
    ns1:PMD_0000077 ex:IDOSP123_fraction_carbon ;
    ns1:PMD_0025999 ex:IDOSP123_some_carbon .

ex:IDOSP123_mass_proportion_chromium a ns1:PMD_0020102 ;
    ns1:PMD_0000077 ex:IDOSP123_fraction_chromium ;
    ns1:PMD_0025999 ex:IDOSP123_some_chromium .

ex:IDOSP123_mass_proportion_nickel a ns1:PMD_0020102 ;
    ns1:PMD_0000077 ex:IDOSP123_fraction_nickel ;
    ns1:PMD_0025999 ex:IDOSP123_some_nickel .

ex:IDOSP123_chem_comp_spec a ns1:PMD_0025002 ;
    ns2:RO_0002351 ex:IDOSP123_fraction_carbon,
        ex:IDOSP123_fraction_chromium,
        ex:IDOS

## Upload data to DSMS as a Knowledge item 

In [8]:
item = KItem(
    name="Inspection documents of steel products demo",
    ktype_id=dsms.ktypes.Dataset,
    apps=[
        {
            "executable": "rdf-upload",
            "title": "data2rdf",
            "additional_properties": {
                "triggerUponUpload": True,
                "triggerUponUploadFileExtensions": [".ttl"],
            },
        }
    ],
    avatar={"include_qr": True},
)
item.attachments = [{"name": "InspectionDocumentsOfSteelProductsAAS.ttl", "content": final_ttl}]
dsms.add(item)
dsms.commit()
print("✓ Knowledge item registered succesfully.")

✓ Knowledge item registered succesfully.
